In [2]:
import requests
import pandas as pd

print("Libraries imported.")

Libraries imported.


In [3]:
df = pd.read_csv("https://www.dropbox.com/scl/fi/zd42ivymo4mxj5yskcoft/city.csv?rlkey=5mqxexw093zxy0t2j917fpkmi&st=8auj9ap3&dl=1");

df.head()

,sum,Organization Location
0,3445350000,"Dublin, Dublin, Ireland, Europe"
1,1893000000,"Bezons, Ile-de-France, France, Europe"
2,1121000000,"Eindhoven, Noord-Brabant, The Netherlands, Europe"
3,7284509737,"London, England, United Kingdom, Europe"
4,1782400000,"Gothenburg, Vastra Gotaland, Sweden, Europe"


In [11]:
import openai
from tqdm import tqdm
import sys
sys.path.append("/Users/franri/Documents/Code/Python/CustomLibs/LLM")
import LanguageModels
import importlib
importlib.reload(LanguageModels);
from LanguageModels import CallLLM

In [49]:
df["GPT_living"] = None;
context_prompt = '''
You are an expert in cost of living estimations. I will give you a city and you have to provide me with the average cost of living in that city.
Your response format must be ONLY the number of the price (in euros), nothing else. Do not hallucinate. 
'''
tqdm.pandas();
for index, row in tqdm(df.iterrows(), total=len(df), desc="Progress"):
    response = CallLLM(context_prompt, "What is the estimated cost of living in " + row["Organization Location"] + "?");
    df.at[index, "GPT_living"] = response[1]["content"];

df.head()

Progress: 100%|██████████| 252/252 [03:05<00:00,  1.36it/s]


,sum,Organization Location,GPT_living,GPT_housing,numbeo_living,numbeo_housing_centre,numbeo_housing_outside
0,3445350000,"Dublin, Dublin, Ireland, Europe",3000,Not found,3753.1,3516.92,2907.14
1,1893000000,"Bezons, Ile-de-France, France, Europe",1500,300000.0,Not found,Not found,Not found
2,1121000000,"Eindhoven, Noord-Brabant, The Netherlands, Europe",1500,400000.0,3493.4,1973.68,1602.38
3,7284509737,"London, England, United Kingdom, Europe",3000,Not found,Not found,Not found,Not found
4,1782400000,"Gothenburg, Vastra Gotaland, Sweden, Europe",1500,500000.0,Not found,Not found,Not found


In [53]:
df["GPT_housing"] = None;
context_prompt = '''
You are an expert in cost of housing estimations. I will give you a city and you have to provide me with an estimation of the average cost of renting a house in that city.
Your response format must be ONLY the number of the price (in euros), nothing else. Do not hallucinate. 
'''
tqdm.pandas();
for index, row in tqdm(df.iterrows(), total=len(df), desc="Progress"):
    response = CallLLM(context_prompt, "What is the estimated cost of housing in " + row["Organization Location"] + "?");
    df.at[index, "GPT_housing"] = response[1]["content"];

df.head()

Progress: 100%|██████████| 252/252 [03:21<00:00,  1.25it/s]


,sum,Organization Location,GPT_living,GPT_housing,numbeo_living,numbeo_housing_centre,numbeo_housing_outside
0,3445350000,"Dublin, Dublin, Ireland, Europe",3000.0,2500,3753.1,3516.92,2907.14
1,1893000000,"Bezons, Ile-de-France, France, Europe",1500.0,1200,Not found,Not found,Not found
2,1121000000,"Eindhoven, Noord-Brabant, The Netherlands, Europe",1500.0,1500,3493.4,1973.68,1602.38
3,7284509737,"London, England, United Kingdom, Europe",3000.0,2500,Not found,Not found,Not found
4,1782400000,"Gothenburg, Vastra Gotaland, Sweden, Europe",1500.0,1500,Not found,Not found,Not found


In [28]:
df.to_csv("temp.csv", index=False)

In [46]:
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

df["numbeo_living"] = "Unknown"

for index, row in tqdm(df.iterrows(), total=len(df), desc="Progress"):
    city = row["Organization Location"].split(", ")[0].replace(" ", "-")
    url = "https://www.numbeo.com/cost-of-living/in/" + city
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        cost_span = soup.find('span', class_='in_other_currency')
        if cost_span:
            text = cost_span.get_text(strip=True)
            match = text.replace("(", "").replace(")", "").replace(",", "").replace("€", "");
            if match:
                try:
                    cost = float(match)
                except ValueError:
                    cost = "Not found"
            else:
                cost = "Not found"
        else:
            cost = "Not found"
        df.at[index, "numbeo_living"] = cost
    else:
        df.at[index, "numbeo_living"] = "Not found"

df["numbeo_housing_centre"] = "Unknown"
df["numbeo_housing_outside"] = "Unknown"

for index, row in tqdm(df.iterrows(), total=len(df), desc="Progress"):
    city = row["Organization Location"].split(", ")[0].replace(" ", "-")
    url = "https://www.numbeo.com/property-investment/in/" + city
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", class_="data_wide_table new_bar_table")

        centre_cost = "Not found"
        outside_cost = "Not found"

        if table:
            rows = table.find_all("tr")
            for r in rows:
                cells = r.find_all("td")
                if len(cells) > 1:
                    label = cells[0].get_text(strip=True)
                    # For "Apartment (3 bedrooms) in City Centre"
                    if "Apartment (3 bedrooms) in City Centre" in label:
                        cost_span = cells[1].find("span", class_="first_currency")
                        if cost_span:
                            text = cost_span.get_text(strip=True)
                            text = text.replace("(", "").replace(")", "").replace(",", "").replace("€", "")
                            text = text.replace(" ", "").replace("\xa0", "")
                            try:
                                centre_cost = float(text)
                            except ValueError:
                                centre_cost = "Not found"
                    # For "Apartment (3 bedrooms) Outside of Centre"
                    elif "Apartment (3 bedrooms) Outside of Centre" in label:
                        cost_span = cells[1].find("span", class_="first_currency")
                        if cost_span:
                            text = cost_span.get_text(strip=True)
                            text = text.replace("(", "").replace(")", "").replace(",", "").replace("€", "")
                            text = text.replace(" ", "").replace("\xa0", "")
                            try:
                                outside_cost = float(text)
                            except ValueError:
                                outside_cost = "Not found"

        df.at[index, "numbeo_housing_centre"] = centre_cost
        df.at[index, "numbeo_housing_outside"] = outside_cost
    else:
        df.at[index, "numbeo_housing_centre"] = "Not found"
        df.at[index, "numbeo_housing_outside"] = "Not found"

df.head()

Progress: 100%|██████████| 252/252 [03:08<00:00,  1.33it/s]


,sum,Organization Location,GPT_living,GPT_housing,numbeo_living,numbeo_housing_centre,numbeo_housing_outside
0,3445350000,"Dublin, Dublin, Ireland, Europe",3000,"As of my last update, the average cost of hous...",3753.1,3516.92,2907.14
1,1893000000,"Bezons, Ile-de-France, France, Europe",1500,300000,Not found,Not found,Not found
2,1121000000,"Eindhoven, Noord-Brabant, The Netherlands, Europe",1500,"400,000",3493.4,1973.68,1602.38
3,7284509737,"London, England, United Kingdom, Europe",3000,"As of my last update, I can't provide real-tim...",Not found,Not found,Not found
4,1782400000,"Gothenburg, Vastra Gotaland, Sweden, Europe",1500,"500,000",Not found,Not found,Not found


In [50]:
def clean_gpt_living(row):
    try:
        text = row["GPT_living"];
        text = text.replace(",", "");
        number = float(text);
        return number
    except:
        return "Not found";

def clean_gpt_housing(row):
    try:
        text = row["GPT_housing"];
        text = text.replace(",", "");
        number = float(text);
        return number
    except:
        return "Not found";

df["GPT_living"] = df.progress_apply(clean_gpt_living, axis=1);
#df["GPT_housing"] = df.progress_apply(clean_gpt_housing, axis=1);

df.head()

100%|██████████| 252/252 [00:00<00:00, 164175.93it/s]


,sum,Organization Location,GPT_living,GPT_housing,numbeo_living,numbeo_housing_centre,numbeo_housing_outside
0,3445350000,"Dublin, Dublin, Ireland, Europe",3000.0,Not found,3753.1,3516.92,2907.14
1,1893000000,"Bezons, Ile-de-France, France, Europe",1500.0,300000.0,Not found,Not found,Not found
2,1121000000,"Eindhoven, Noord-Brabant, The Netherlands, Europe",1500.0,400000.0,3493.4,1973.68,1602.38
3,7284509737,"London, England, United Kingdom, Europe",3000.0,Not found,Not found,Not found,Not found
4,1782400000,"Gothenburg, Vastra Gotaland, Sweden, Europe",1500.0,500000.0,Not found,Not found,Not found


In [52]:
df.head(15)

,sum,Organization Location,GPT_living,GPT_housing,numbeo_living,numbeo_housing_centre,numbeo_housing_outside
0,3445350000,"Dublin, Dublin, Ireland, Europe",3000.0,Not found,3753.1,3516.92,2907.14
1,1893000000,"Bezons, Ile-de-France, France, Europe",1500.0,300000.0,Not found,Not found,Not found
2,1121000000,"Eindhoven, Noord-Brabant, The Netherlands, Europe",1500.0,400000.0,3493.4,1973.68,1602.38
3,7284509737,"London, England, United Kingdom, Europe",3000.0,Not found,Not found,Not found,Not found
4,1782400000,"Gothenburg, Vastra Gotaland, Sweden, Europe",1500.0,500000.0,Not found,Not found,Not found
5,1609200000,"Schaffhausen, Schaffhausen, Switzerland, Europe",2500.0,800000.0,Not found,Not found,Not found
6,600000000,"Alphen Aan Den Rijn, Zuid-Holland, The Netherl...",1500.0,350000.0,Not found,Not found,Not found
7,600000000,"Pesaro, Marche, Italy, Europe",1200.0,250000.0,Not found,Not found,Not found
8,650000000,"Louvain-la-neuve, Vlaams-Brabant, Belgium, Europe",1200.0,300000.0,Not found,Not found,Not found
9,2534849994,"Paris, Ile-de-France, France, Europe",3000.0,12000.0,3758.7,3015.0,2059.26
